<a href="https://colab.research.google.com/github/JeonCollin/mask-detection/blob/main/mask_evaluation%EC%9D%98_%EC%A7%84%EC%A7%9C_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Evaluation**


*   load custom images and classify wearing a mask or not
*   show the test images and prediction or/and scores

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.models as models
from torchvision import transforms, datasets
from torchvision.transforms.functional import to_pil_image
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from google.colab.patches import cv2_imshow
import os
import cv2
import numpy as np
import face_recognition

import matplotlib.pyplot as plt
%matplotlib inline
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from torchvision.transforms.transforms import CenterCrop
from torchvision.transforms.transforms import Resize

transforms = transforms.Compose([
transforms.Resize(112),
transforms.ToTensor(),
])

In [ ]:
# 1. prepare data
test_path = "/content/drive/MyDrive/detect_mask/test"
test_data = ImageFolder(root=test_path, transform=transforms)
test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

#image = face_recognition.load_image_file("your_file.jpg")
#face_locations = face_recognition.face_locations(image)


In [ ]:
# 2. load model
model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.BatchNorm1d(num_ftrs),
    nn.Linear(num_ftrs, 1)
)
model.load_state_dict(torch.load("/content/drive/MyDrive/detect_mask/result20240604-01/model.pth"))
model = model.to(device)
model.eval()

In [ ]:
# 3. perform evaluation
def evaluate(model, dataloader):
    all_labels = []
    all_preds = []

    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device).float()
        with torch.no_grad():
            outputs = model(inputs)
            preds = torch.sigmoid(outputs) > 0.5
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

            # Display the image with prediction
            img = inputs.cpu().squeeze(0)
            img = to_pil_image(img)
            plt.imshow(img)
            plt.title(f'Predicted: {"Mask" if preds.item() else "No Mask"}')
            plt.show()

    return all_labels, all_preds

all_labels, all_preds = evaluate(model, test_loader)